In [ ]:
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
 
# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
import os
import MeCab
import re
import subprocess
import sys
path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -b 819200"
m = MeCab.Tagger(path)
noun_list = ''
from google.colab import drive
drive.mount('/content/drive')
for bunrui in range(1,51):
  lssentence = " ".join(['ls','/content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/%02d/*.nml' % bunrui])
  output = subprocess.getoutput(lssentence)
  tolists = output.split('\n')
  for tolist in tolists:
    print(tolist)
    text_data = open(tolist, "rb").read()
    decoded_text = text_data.decode('utf-8')
    text = re.sub('\n\n', '\n', decoded_text)
    with open(tolist + '.wwakati', 'a') as f:
      parses = m.parse(text)
      if parses is None:
        tolistformecab = re.sub(" ", "\ ", tolist)
        mecabsentence = " ".join(['cat', tolistformecab, '|', 'mecab', path])
        parses = subprocess.getoutput(mecabsentence)
      t_parses = parses.split(os.linesep)
      for t_parse in t_parses:
        t_lists = t_parse.split('\t')
        term = t_lists[0] + ' '
        if t_lists[0] == 'EOS' or t_lists[0] == '' or t_lists[0] == 'と' or t_lists[0] == 'の':
          term = ''
          continue
        for hinsi in ['記号', '助詞,連体化', '助詞,副詞化', '助詞,係助詞', '助詞,接続助詞', '助詞,並立助詞', '助詞,副助詞／並立助詞／終助詞', '助詞,格助詞,引用', '助詞,格助詞,一般,*,*,*,へ,ヘ,エ', '助詞,格助詞,一般,*,*,*,に,ニ,ニ', '助詞,格助詞,一般,*,*,*,を,ヲ,ヲ', '助詞,格助詞,一般,*,*,*,で,デ,デ', '助詞,格助詞,一般,*,*,*,が,ガ,ガ', '助詞,格助詞,一般,*,*,*,から,カラ,カラ', '助詞,副助詞,*,*,*,*,まで,マデ,マデ', '連体詞', '接頭詞,名詞接続', '特殊', '形容詞,自立,*,*,形容詞・アウオ段,基本形,ない,ナイ,ナイ', '接頭詞,数接続', '名詞,数', '名詞,接尾', '名詞,非自立,副詞可能', '名詞,非自立,助動詞語幹', '名詞,接尾,助数詞', '名詞,代名詞', '助動詞', '名詞,非自立,一般', '動詞,非自立', '動詞,自立,*,*,サ変・スル', '動詞,接尾', '動詞,自立,*,*,一段,基本形,できる,デキル,デキル', '動詞,自立,*,*,五段・ラ行,基本形,係る,カカル,カカル', '助詞,格助詞,連語,*,*,*,に']:
          if t_lists[1].find(hinsi) > -1:
            term = ''
        for stopword in ['ヶ所', 'ヵ所', 'カ所', '箇所', 'ヶ月', 'ヵ月', 'カ月', '箇月', '十', '百', '千', '万', '億', '兆', 'もの', '明治', '大正', '昭和', '平成', '令和']:
          if t_lists[0].find(stopword) > -1:
            term = ''
        noun_list = noun_list + term
      f.write(noun_list)
      noun_list = ''
f.close()

In [ ]:
!find /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/ -name "*.wwakati" -type f -printf "%p,%s\n" > /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/bunruiallw.txt
# 法令ファイルには既施行と未施行が重複している場合があるため、ファイル名の前半部分を使って重複を取り除き、bunruiwall-sort-uniq2.csvを作成する。

In [ ]:
import os
import sys
import re
import subprocess
from google.colab import drive
drive.mount('/content/drive')
f=open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/bunruiwall-sort-uniq2.csv')
lines=f.readlines()
f.close()
bfs = []
sums= []
nums = []
mods = []
bunrui = 0
bunruii = 1
#以下の1690000は、2160000ではメモリ不足になる場合に使用している。通常は2160000でよい。
for line in lines:
  bunrui = int(line[0:2])
  fn = line[3:64]
  size = int(line[65:].replace("\n", ""))
  bfs.append((bunrui, fn, size))
sums = [0, 0]
nums = [0]
mods = [0]
bunruilens = [0]
bunruilen = 0
for bfsi in bfs:
  if bfsi[0]  == bunruii:
    sums[bunruii] = sums[bunruii] + bfsi[2]
    bunruilen += 1
  elif sums[bunruii]//1690000  == 0:
    nums.append(1)
    mods.append(0)
    sums.append(bfsi[2])
    bunruilens.append(bunruilen)
    print(bunruii, 1, 0, bunruilen)
    bunruilen = 1
    bunruii = bfsi[0]
  else:
    nums.append(sums[bunruii]//1690000 + 1)
    mods.append(((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1)
    sums.append(bfsi[2])
    bunruilens.append(bunruilen)
    print(bunruii, sums[bunruii], sums[bunruii]//1690000 + 1, ((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1, bunruilen)
    bunruilen = 1
    bunruii = bfsi[0]
if sums[bunruii]//1690000  == 0:
  nums.append(1)
  mods.append(0)
  bunruilens.append(bunruilen)
  print(bunruii, 1, 0)
else:
  nums.append(sums[bunruii]//1690000 + 1)
  mods.append(((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1)
  bunruilens.append(bunruilen)
  print(bunruii, sums[bunruii]//1690000 + 1, ((sums[bunruii] % -1690000)//(nums[bunruii] - 1)) * -1, bunruilen)
print(len(sums), len(nums), len(mods), len(bunruilens))
print(bunruilens)

In [ ]:
wakati = ['']
wakatilist = []
wakatilistbw = []
numw = 0
sumfw = 0
sumbw = 0
t = 0
outlist = [11]
def writeallwakati(bunruii, numw, wakatilist):
  fn = '/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/01-50wakatiall/%02d-%02dw.wwakatiall' % (bunruii, numw)
  with open(fn, 'a') as fa:
    for tolist in wakatilist:
      if not tolist == '':
        text_data = open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/%02d/' % bunruii + tolist[0], "rb").read()
        text = text_data.decode('utf-8')
      fa.write(text)
 
for bunruii in range(1, 51):
  start = sum(bunruilens[0:bunruii])
  end = sum(bunruilens[0:bunruii + 1]) - 1
  print(bunruii, start, end)
  for t1, bfsi in enumerate(bfs[start:end + 1]):
    sumfw = sumfw + int(bfsi[2])
    if sumfw > 1590000:
      print('reversed', bunruii, sumfw)
      wakati.append((bunruii, numw, wakatilist))
      writeallwakati(bunruii, numw, wakatilist)
      wakatilist = []
      wakatilistbw = []
      numw += 1
      sumfw = 0
      for t2, bfsbw in enumerate(reversed(bfs[start:start + t1])):
        sumbw = sumbw + int(bfsbw[2])
        print(sumbw, int(mods[bunruii])/(int(nums[bunruii]) - 1))
        if sumbw > int(mods[bunruii])/(int(nums[bunruii]) - 1):
          sumfw = sumbw - int(bfsbw[2]) + int(bfsi[2]) 
          print('sumfw is %d' % sumfw)
          wakatilist = wakatilistbw[:]
          wakatilistbw = []
          sumbw = 0
          break
        wakatilistbw.append((bfsbw[1],bfsbw[2]))
        print(t2, sumbw)
      else:
        sumfw = sumbw + int(bfsi[2]) 
        print('sumbw is %d' % sumbw)
        wakatilist = wakatilistbw[:]
        wakatilistbw = []
        sumbw = 0
    wakatilist.append((bfsi[1],bfsi[2]))
    print(t1, sumfw)
  wakati.append((bunruii, numw, wakatilist))
  writeallwakati(bunruii, numw, wakatilist)
  wakatilist = []
  wakatilistbw = []
  sumfw = 0
  numw = 0
  sumbw = 0


In [ ]:
import os
import sys
import subprocess
%cd /content/
!wget https://github.com/facebookresearch/fastText/archive/0.2.0.zip
!unzip 0.2.0.zip
%cd fastText-0.2.0
!make
%cd /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/01-50wakatiall
lswakatiall = " ".join(['ls','*.wwakatiall'])
output = subprocess.getoutput(lswakatiall)
print(output)
%cd /content/
for fni in output.split('\n'):
  print(fni)
  fno = fni + ".model"
  fni = "/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/01-50wakatiall/" + fni
  fasttextcommand = ["/content/fastText-0.2.0/fasttext", "skipgram", "-dim", "128", "-minCount", "1", "-input", fni, "-output", fno]
  subprocess.call(fasttextcommand)
  rmfn = fno + '.bin'
  cpfn = fno + '.vec'
  rmcommand = ["rm", rmfn]
  cpcommand = ["cp", cpfn, "/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/"]
  subprocess.call(rmcommand)
  subprocess.call(cpcommand)

In [ ]:
import os
import sys
import glob
import math
import json
from pathlib import Path 
import pickle
from google.colab import drive
drive.mount('/content/drive')
for fni in output.split('\n'):
  print(fni)
  term_vec = []
  fnmodelvec = fni + '.model.vec'
  fntermvec = fni + '.term_vec.pkl'
  with open(fnmodelvec, 'r') as f:
    next(f)
    for i, line in enumerate(f):
        ents = line.split()
        term = ' '.join(ents[:-128])
        vec = list(map(float, ents[-128:]))
        term_vec.append([term, i, vec])
    open(fntermvec, 'wb').write(pickle.dumps(term_vec))


In [ ]:
from __future__ import print_function
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, concatenate
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers.core import Dropout, Reshape, Permute, RepeatVector, Flatten, Lambda
from keras.layers.normalization import BatchNormalization as BN
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, Adam, Adagrad, Nadam, SGD, Adadelta, Adamax
from keras.utils.data_utils import get_file
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
import numpy as np
import random
import sys
import glob
from time import gmtime, strftime
import copy
import pickle
import json
import tensorflow as tf
import math
import subprocess
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/01-50wakatiall/
lswakatiall = " ".join(['ls','*.wwakatiall'])
output = subprocess.getoutput(lswakatiall)
%cd /content/
rng = 5
for fni in output.split('\n'):
  dataset = []
  term_vec = {}
  ansidx = {}
  fntermvec = fni + '.term_vec.pkl'
  fnwakatiall = '/content/drive/MyDrive/Colab Notebooks/houreibunrui20210510/01-50wakatiall/' + fni
  fndataset = fni + 'dataset.pkl'
  term_vec_lists = pickle.loads(open(fntermvec, 'rb').read())
  for term_vec_list in term_vec_lists:
    term_vec[term_vec_list[0]] = term_vec_list[2]
    ansidx[term_vec_list[0]] = term_vec_list[1]
  with open(fnwakatiall, 'r') as f:
    for fi, line in enumerate(f):
      terms = line.split()
      for cur in range(rng, len(terms) - rng, 1):
        try:
          head = list(map(lambda x:term_vec[x], terms[cur-rng:cur]))
          ans  = terms[cur]
          tail = list(map(lambda x:term_vec[x], terms[cur+1:cur+rng]))
          pure_text = terms[cur-rng:cur+rng]
          dataset.append( (head, ansidx[ans], tail, pure_text) )
        except KeyError as e:
          pass
  print(fni," data set is %d" % len(dataset))
  open(fndataset, 'wb').write(pickle.dumps(dataset))

In [ ]:
import pickle
import numpy as np
import random
import os
import sys
import subprocess
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, concatenate
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dropout, Reshape, Permute, RepeatVector, Flatten, Lambda
from tensorflow.keras.layers import BatchNormalization as BN
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import RMSprop, Adam, Adagrad, Nadam, SGD, Adadelta, Adamax
from tensorflow.keras.callbacks import ReduceLROnPlateau

%cd /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/01-50wakatiall/
lswakatiall = " ".join(['ls','*.wwakatiall'])
output = subprocess.getoutput(lswakatiall)
print(output)
%cd /content/
rng = 5
def build_model(maxlen=None, out_dim=None, in_dim=128):
  print('Build model...')
  model = Sequential()
  model.add(GRU(128*rng*2, return_sequences=False, input_shape=(maxlen, in_dim)))
  model.add(BN())
  model.add(Dense(out_dim))
  model.add(Activation('linear'))
  model.add(Activation('sigmoid'))
  optimizer = Adam()
  model.compile(loss='binary_crossentropy', optimizer=optimizer)
  return model
print("importing data from algebra...")
for fni in output.split('\n'):
  fndatasetpkl = fni + 'dataset.pkl'
  datasets = pickle.loads(open(fndatasetpkl, 'rb').read())
  if len(datasets) == 0:
    continue
  sentences = []
  answers   = []
  term_vec = {}
  ansidx = {}
  random.shuffle(datasets)
  for dbi, series in enumerate(datasets):
    head, ans, tail, pure_text = series 
    head.extend(tail)
    sentences.append(np.array(head))
    answers.append(ans)
  print('nb sequences:', len(sentences))
  fntermvecpkl = fni + '.term_vec.pkl'
  term_vec_lists = pickle.loads(open(fntermvecpkl, 'rb').read())
  for term_vec_list in term_vec_lists:
    term_vec[term_vec_list[0]] = term_vec_list[2]
    ansidx[term_vec_list[0]] = term_vec_list[1]
  term_vec_lists = []
  term_vec_list = []
  print('Vectorization...')
  print(len(term_vec))
  X = np.zeros((len(sentences), len(sentences[0]), 128), dtype=np.float64)
  print(len(sentences),len(answers))
  y = np.zeros((len(sentences), len(term_vec)), dtype=np.int)
  for i, sentence in enumerate(sentences):
    if i%10000 == 0:
      print("building training vector... iter %d"%i)
    for t, vec in enumerate(sentence):
      X[i, t, :] = vec
    y[i, answers[i]] = 1
  model = build_model(maxlen=len(sentences[0]), in_dim=128, out_dim=len(term_vec))
  for iteration in range(1, 31):
    print()
    print('-' * 50)
    print('Iteration', iteration)
 
    model.fit(X, y, batch_size=128, epochs=1)
    tf.keras.backend.clear_session()
    gc.collect()
    if iteration == 30:
      MODEL_NAME = './models' + fni + "/snapshot.%09d.model" % iteration
      model.save(MODEL_NAME)

  del X
  del y
  del model
